# Flux Scope

Experiment to characterise the distortions of flux pulses due to the imperfect signal lines, following chapter 4.4.3 in https://www.research-collection.ethz.ch/handle/20.500.11850/153681

## 0. General Imports and Definitions

### 0.1 Python Imports 

In [ ]:
# LabOne Q:
from laboneq.contrib.example_helpers.example_notebook_helper import create_device_setup

# Helpers:
from laboneq.contrib.example_helpers.plotting.plot_helpers import *
from laboneq.simple import *

## 1. Device Setup

In [ ]:
# create device setup
device_setup = create_device_setup(generation=2)
use_emulation = True

In [ ]:
#  signal map for qubits
def map_qubit(qubit_id):
    return {
        "drive": f"/logical_signal_groups/q{qubit_id}/drive_line",
        "flux": f"/logical_signal_groups/q{qubit_id}/flux_line",
        "measure": f"/logical_signal_groups/q{qubit_id}/measure_line",
        "acquire": f"/logical_signal_groups/q{qubit_id}/acquire_line",
    }

## 2. Experiment Definition

In [ ]:
# Pulse definitions

# qubit excitation pulse - amplitude such that if pulse is resonant, results in pi rotation
x180 = pulse_library.gaussian(uid="x180", length=20e-9, amplitude=0.66)

# flux pulse - constant length and amplitude
flux_pulse = pulse_library.const(uid="flux_pulse", length=400e-9, amplitude=0.5)

# readout drive pulse
readout_pulse = pulse_library.const(uid="readout_pulse", length=250e-9, amplitude=1.0)
# readout weights for integration
readout_weighting_function = pulse_library.const(
    uid="readout_weighting_function", length=200e-9, amplitude=1.0
)

# assuming all calibration settings are already correct

In [ ]:
# define sweep parameters

# qubit excitation pulse frequency sweep
start_freq = 40e6
stop_freq = 200e6
count_freq = 11

sweep_frequency = LinearSweepParameter(
    uid="qubit_frequency", start=start_freq, stop=stop_freq, count=count_freq
)

# sweep delay between start of flux pulse and start of qubit excitation pulse
start_delay = 0
stop_delay = flux_pulse.length - x180.length
count_delay = 11

sweep_delay = LinearSweepParameter(
    uid="delay", start=start_delay, stop=stop_delay, count=count_delay
)

# define number of averages
average_exponent = 10  # used for 2^n averages, n=average_exponent, maximum: n = 19

### 2.1 Pulse Sequence

In [ ]:
# Create Experiment
exp = Experiment(
    "Flux Scope",
    signals=[
        ExperimentSignal("drive"),
        ExperimentSignal("flux"),
        ExperimentSignal("measure"),
        ExperimentSignal("acquire"),
    ],
)

## experimental pulse sequence
# outer sweep - qubit excitation frequency

# real-time acquisition loop in integration mode
with exp.acquire_loop_rt(
    uid="shots",
    count=pow(2, average_exponent),
    averaging_mode=AveragingMode.CYCLIC,
    acquisition_type=AcquisitionType.INTEGRATION,
):
    with exp.sweep(uid="frequency_sweep", parameter=sweep_frequency):
        # inner sweep - delay between start of qubit excitation pulse and start of flux pulse
        with exp.sweep(uid="sweep", parameter=sweep_delay):
            # flux pulse
            with exp.section(uid="qubit_excitation"):
                exp.play(signal="flux", pulse=flux_pulse)  # qubit detuning
                exp.delay(signal="drive", time=sweep_delay)  # delay is swept
                exp.play(signal="drive", pulse=x180)  # qubit excitation
            # readout and data acquisition
            with exp.section(uid="qubit_readout", play_after="qubit_excitation"):
                exp.play(signal="measure", pulse=readout_pulse)
                exp.acquire(
                    signal="acquire",
                    handle="ac_0",
                    kernel=readout_weighting_function,
                )
            # relax time after readout - for signal processing and qubit relaxation to ground state
            with exp.section(uid="relax", play_after="qubit_readout"):
                exp.delay(signal="measure", time=1e-6)

In [ ]:
# define experiment calibration - sweep over qubit drive frequency
exp_calib = Calibration()
exp_calib["drive"] = SignalCalibration(
    oscillator=Oscillator(
        frequency=sweep_frequency,
        modulation_type=ModulationType.HARDWARE,
    )
)

### 2.2 Run the Experiment and Plot the Measurement Results and Pulse Sequence

In [ ]:
# set calibration and signal map for qubit 0
exp.set_calibration(exp_calib)
exp.set_signal_map(map_qubit(0))

# create a session and connect to it
session = Session(device_setup=device_setup)
session.connect(do_emulation=use_emulation)

# run experiment on qubit 0
my_results = session.run(exp)

In [ ]:
# Plot simulated output signals
plot_simulation(session.compiled_experiment, start_time=0, length=10e-6)

In [ ]:
# plot measurement results
plot_result_3d(my_results, "ac_0")

In [ ]:
# use pulse sheet viewer to display the pulse sequence - only recommended for small number of averages and sweep steps to avoid performance issues
show_pulse_sheet("Flux Scope Experiment", session.compiled_experiment)

In [ ]:
# set calibration and signal map for qubit 1
exp.set_calibration(exp_calib)
exp.set_signal_map(map_qubit(1))

# create a session and connect to it
session = Session(device_setup=device_setup)
session.connect(do_emulation=use_emulation)

# run experiment on qubit 1
my_results = session.run(exp)